<a href="https://colab.research.google.com/github/Python-Is-Long/Teaching/blob/main/Webscraping_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2020-7-7

In [ ]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# Goal:

Get the airline data from the tables on this webpage: https://www.iata.org/en/publications/directories/code-search/

In [ ]:
url = "https://www.iata.org/en/publications/directories/code-search/"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
soup


<!DOCTYPE html>

<html class="on-page-editor" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>IATA - Codes - Airline and Location Codes Search</title>
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/site.webmanifest" rel="manifest"/>
<link color="#1e32fa" href="/safari-pinned-tab.svg" rel="mask-icon"/>
<meta content="#da532c" name="msapplication-TileColor"/>
<meta content="#ffffff" name="theme-color"/>
<meta content="Official directory of IATA codes used by airlines and airports, including IATA 3-letter and 2-letter codes." name="description">
<meta content="IATA Codes, airline codes, airport codes, location identifier, airline name, airport name, IATA Airline codes, IA

The data are under the table tag, so we need to get it from the HTML document

In [ ]:
table = soup.find("table")
print(table.prettify())

<table class="datatable">
 <thead>
  <tr>
   <td>
    Company name
   </td>
   <td>
    Country / Territory
   </td>
   <td>
    2-letter code
   </td>
   <td>
    Accounting code (PAX)
   </td>
   <td>
    Airline prefix code
   </td>
  </tr>
 </thead>
 <tbody>
  <tr>
   <td>
    21 Air LLC
   </td>
   <td>
    UNITED STATES OF AMERICA
   </td>
   <td>
    2I*
   </td>
   <td>
   </td>
   <td>
    681
   </td>
  </tr>
  <tr>
   <td>
    40-Mile Air, Ltd.
   </td>
   <td>
    UNITED STATES OF AMERICA
   </td>
   <td>
    Q5*
   </td>
   <td>
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td>
    748 Air Services
   </td>
   <td>
    Kenya
   </td>
   <td>
    H4*
   </td>
   <td>
    874
   </td>
   <td>
    874
   </td>
  </tr>
  <tr>
   <td>
    8165343 Canada Inc. dba Air Canada Rouge
   </td>
   <td>
    Canada
   </td>
   <td>
    RV
   </td>
   <td>
   </td>
   <td>
   </td>
  </tr>
  <tr>
   <td>
    9 Air Co Ltd
   </td>
   <td>
    People's Republic of China
   </td>
   <td>
   

In [ ]:
df = pd.read_html(str(table))[0]
df

,Company name,Country / Territory,2-letter code,Accounting code (PAX),Airline prefix code
0,21 Air LLC,UNITED STATES OF AMERICA,2I*,NaN,681.0
1,"40-Mile Air, Ltd.",UNITED STATES OF AMERICA,Q5*,NaN,NaN
2,748 Air Services,Kenya,H4*,874.0,874.0
3,8165343 Canada Inc. dba Air Canada Rouge,Canada,RV,NaN,NaN
4,9 Air Co Ltd,People's Republic of China,AQ,902.0,902.0


Now that we know how to convert the table in the first page into a dataframe, we can make an automated process to scrape every page and combine all the information in a single dataframe.

In [ ]:
n_pages = 218 #total pages on the website
df_list = [] #use this to store individual tables for different pages (we'll glue them together in the end)
for i in tqdm(range(1,n_pages+1)):
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'} #use this to make the website think we are not a bot
  url = "https://www.iata.org/en/publications/directories/code-search/?airline.page={page}&airline.search=".format(page=i)
  page = requests.get(url, headers=headers)
  soup = BeautifulSoup(page.content, 'html.parser')
  table = soup.find("table")
  df_list += [pd.read_html(str(table))[0]]

In [ ]:
df_combined = pd.concat(df_list, axis=0)
df_combined

,Company name,Country / Territory,2-letter code,Accounting code (PAX),Airline prefix code
0,21 Air LLC,UNITED STATES OF AMERICA,2I*,NaN,681.0
1,"40-Mile Air, Ltd.",UNITED STATES OF AMERICA,Q5*,NaN,NaN
2,748 Air Services,Kenya,H4*,874.0,874.0
3,8165343 Canada Inc. dba Air Canada Rouge,Canada,RV,NaN,NaN
4,9 Air Co Ltd,People's Republic of China,AQ,902.0,902.0
...,...,...,...,...,...
2,ZanAir Limited,"Tanzania, United Republic of",B4,NaN,NaN
3,"Zapways, Inc.",UNITED STATES OF AMERICA,R1,407.0,NaN
4,"Zhejiang Loong Airlines Co., Ltd",People's Republic of China,GJ,891.0,891.0
0,Zimex Aviation Ltd,Switzerland,XM,NaN,NaN


In [ ]:
%%time
df_combined.to_csv("IATA scraped data 2020-7-7.csv", encoding="utf-8-sig")

CPU times: user 13.4 ms, sys: 1.02 ms, total: 14.4 ms
Wall time: 18.8 ms


In [ ]:
!ls -l *.csv

-rw-r--r-- 1 root root 58263 Jul  7 16:59 'IATA scraped data 2020-7-7.csv'
